# Finding home statistical area - samples

In [2]:
import geopandas as gp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Point
import nafot
import time
import random
%matplotlib inline

#### Get the statistical area GeoDataFrame

In [1]:
gdf = nafot.gdf.copy()

NameError: name 'nafot' is not defined

#### Loading location data

In [3]:
loc_data = pd.read_csv(r'C:\Users\user\Documents\דרופבוקס\Data - cellular db\with stat area\sample2_100M_with_stat.csv')

# Remove records without stat area
loc_data.dropna(inplace=True)

In [4]:
loc_data[~loc_data.stat_area_id.isnull()].shape

(92832600, 7)

Getting only calls at night - 18:00 - 08:00

In [5]:
#defining night hours - 18:00 - 08:00 by half hour index
night_hours = list(range(16+1))
night_hours.extend(list(range(36,47+1)))

night_data = loc_data[loc_data['halfhouridx'].isin(night_hours)]

Getting the most visited statistical areas

In [6]:
imsi_list = list(pd.unique(night_data['imsi']))
stat_areas_list = pd.unique(night_data['stat_area_id'])

# Creating a dict to save n, x1, x2 for each imsi - key: imsi, value:
# ((n, x1, x2), (first stat aarea name, second stat area name))
poll_dist = {}

for curr_imsi in imsi_list:
    # creating a histogram of calls from each stat_area
    stat_areas_count = {stat_area_id : 0 for stat_area_id in stat_areas_list}
       
    # creating a df of the relevant imsi
    imsi_data = night_data[night_data['imsi'] == curr_imsi]
    
    for stat_area in imsi_data['stat_area_id']:
        stat_areas_count[stat_area] += 1
    
    # getting the number of calls at night
    n = len(imsi_data.index)
    
    #Removing imsi with less then 50 calls at night
    if n > 50:
        # getting the stat_area with largest number of calls
        first_stat_area_name = max(stat_areas_count, key = stat_areas_count.get)
        first_stat_area = (first_stat_area_name, stat_areas_count.pop(first_stat_area_name))

        # getting the stat_area with second largest number of calls
        second_stat_area_name = max(stat_areas_count, key = stat_areas_count.get)
        second_stat_area = (second_stat_area_name, stat_areas_count.pop(second_stat_area_name))

        # getting the number of calls in the first and second stat_area.
        x1 = first_stat_area[1]
        x2 = second_stat_area[1]

        # Saving n, x1 and x2 for each imsi
        poll_dist[curr_imsi] = ((n, x1, x2), (first_stat_area[0], second_stat_area[0]))

Running a simulation for each imsi to find the cutoff and decide a home stat area

In [47]:
# Creating a dict to save the home stat area
home_stat_area = {}

# Defining a threshold and significance level (in precents) 
significance = 25 # %

for curr_imsi in poll_dist:
    # creating a list for the proportions differences
    diffs = []
    
    # getting the estimated values
    n, x1, x2 = poll_dist[curr_imsi][0]
    diff = x1-x2
    # p - the avarage of p1 and p2
    p = (x1 + x2)/(2*n)
    
    # The simulation
    for i in range(100):
        X = np.random.multinomial(n, [p, p, 1 - 2*p])
        diffs.append(X[0] - X[1])
        
    # The cutoffs are the 2.5 and 97.5 percentiles 
    upper_cutoff = np.percentile(diffs, 100 - significance/2)
    lower_cutoff = np.percentile(diffs, significance/2)
    
    # if the difference between x1 and x2 in the data is higher than the upper cutoff
    # then the first stat_area is the home stat area
    if diff > upper_cutoff:
        home_stat_area[curr_imsi] = poll_dist[curr_imsi][1][0]
    
    # if the difference between x1 and x2 in the data is lower than the lower cutoff
    # then the second stat_area is the home stat area
    elif diff < lower_cutoff:
        home_stat_area[curr_imsi] = poll_dist[curr_imsi][1][1]
    
    # if the difference between x1 and x2 in the data is between the lower and uppper cutoffs
    # then we cannot detrmaine a home stat area 
    else:
        home_stat_area[curr_imsi] = 'NotDetermined'

Saving the data about home stat_area

In [48]:
home_stat_area_data = pd.DataFrame(list(home_stat_area.items()), columns = ['imsi', 'home_stat_area']).set_index('imsi')

In [49]:
home_stat_area_data[home_stat_area_data['home_stat_area'] != 'NotDetermined'].count()

home_stat_area    5621
dtype: int64

In [50]:
home_stat_area_data.to_csv('./home_stat_area_data_sample2_100M.csv')

Only calls at active hours

In [26]:
active_hours = list(range(12,47+1))

active_hours_data = loc_data[loc_data['halfhouridx'].isin(active_hours)]

Creating the meeting matrix

In [27]:
# Get a list of the stat_area's ids
stat_ids = list(gdf.index.values)

#Creating the meeting matrix
matrix_A = np.zeros((len(stat_ids), len(stat_ids)), dtype = float)

for i, curr_stat in enumerate(stat_ids):
    #get all the imsi which thier home stat_area is the current stat_area
    home_imsi = home_stat_area_data[home_stat_area_data['home_stat_area'] == curr_stat].index
    
    for curr_imsi in home_imsi:
        # creating a histogram of calls from each stat_area
        stat_area_count = {stat_area_id : 0 for stat_area_id in stat_ids}
        
        # creating a DF of the relevant imsi
        imsi_data = active_hours_data[active_hours_data['imsi'] == curr_imsi]
    
        for stat_area_id in imsi_data['stat_area_id']:
            stat_area_count[stat_area_id] += 1
            
        # getting the number of calls
        n = len(imsi_data.index)
        # if there are no calls on active hours
        if n == 0:
            continue
    
        for visiting_stat_area in stat_area_count:
            # row: i - the current home stat_area
            # column: the visiting stat_area
            # value: adding the propotion of calls from visiting stat_area for current imsi 
            matrix_A[i, stat_ids.index(visiting_stat_area)] += (stat_area_count[visiting_stat_area] / n)

In [7]:
matrix_A.shape

NameError: name 'matrix_A' is not defined

In [29]:
relevant_imsi = home_stat_area_data[home_stat_area_data['home_stat_area'] != 'NotDetermined'].index.values
pd.unique(loc_data[(loc_data.imsi.isin(relevant_imsi))].stat_area_id).size

1608

In [31]:
np.save('../data/matrix/sample1_raw_mat', matrix_A)

Normalizing the meeting matrix

In [6]:
# Normalizing the meeting matrix by deviding each row by it's sum (the number of residents)
matrix_B = matrix_A.copy()
for i in range(matrix_B.shape[0]):
    if matrix_B[i].sum() > 0:
        matrix_B[i] = matrix_B[i] / (matrix_B[i].sum())

# Creating a DF of the meetings
# meeting_data = pd.DataFrame(normalized_meeting_matrix, index = stat_ids, columns = stat_ids)
# meeting_data

NameError: name 'matrix_A' is not defined

Checking the rows sum - suppose to be 1

In [39]:
total_sum = 0
for i, stat_area in enumerate(stat_ids):
#     print ('{} : {}'.format(stat_area,sum(normalized_meeting_matrix[i])))
    total_sum += sum(meeting_matrix[i])
print (total_sum)

6156.0


### Creating the meeting probabilities maxtrix

In [49]:
no_pop = 0
no_pop2 = 0

# Multiply each row by the population of the stat_area
matrix_C = matrix_B.copy()

for i in range(matrix_C.shape[0]):
    # Get the stat_area population (the data is in thousands)
    stat_area_pop = gdf.iloc[i].pop_thou * 1000
    
    if np.isnan(stat_area_pop):
        stat_area_pop = 0
        no_pop+=1 
        if matrix_C[i].sum() > 0:
            no_pop2+=1
    else:
        # Multiply the row by the relevant population
        matrix_C[i] *= stat_area_pop

print ('Number of stat areas without population data (not home stat area): {}'.format(no_pop))
print ('Number of stat areas without population data (home stat area): {}'.format(no_pop2))

Number of stat areas without population data (not home stat area): 771
Number of stat areas without population data (home stat area): 173


In [50]:
matrix_D = matrix_C.copy()

# Normlize each column (sum of 1)
for i in range(matrix_C.shape[1]):
    if matrix_D[:,i].sum() > 0:
        matrix_D[:,i] = matrix_D[:,i] / (matrix_D[:,i].sum())

In [179]:
#OLD
meeting_probs = np.zeros((len(stat_ids), len(stat_ids)), dtype = float)

for i in range(meeting_probs.shape[0]):
    for j in range(meeting_probs.shape[1]):
        meeting_probs[i,j] = np.dot(meeting_matrix2[i], meeting_matrix2[j])

In [85]:
from tqdm import tqdm_notebook as tqdm

In [94]:
meeting_matrix_P = np.zeros((len(stat_ids), len(stat_ids)), dtype = float)

for i in tqdm(range(meeting_matrix_P.shape[0])):
    for j in range(meeting_matrix_P.shape[1]):
        meeting_matrix_P[i,j] = np.sum(matrix_B[i]*matrix_B[j]*matrix_D[j])

#         for k in range(len(stat_ids)):
#             meeting_matrix_P[i,j] += (matrix_B[i,k]*matrix_B[j,k]*matrix_D[j,k])

In [96]:
meeting_matrix_P1 = meeting_matrix_P.copy()

In [ ]:
meeting_matrix_P2 = np.zeros((len(stat_ids), len(stat_ids)), dtype = float)

for i in tqdm(range(meeting_matrix_P.shape[0])):
    for j in range(meeting_matrix_P.shape[1]):
        for k in range(len(stat_ids)):
            meeting_matrix_P2[i,j] += (matrix_B[i,k]*matrix_B[j,k]*matrix_D[j,k])

In [88]:
a = np.array([[1,2,3], [3,4,5],[7,8,9]])
b = a.copy()
b *= 2

In [89]:
a

array([[1, 2, 3],
       [3, 4, 5],
       [7, 8, 9]])

In [90]:
b

array([[ 2,  4,  6],
       [ 6,  8, 10],
       [14, 16, 18]])

In [92]:
np.sum(a[0]*b[0]*a[1])

128

In [93]:
c = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-93-a45ec2eb1754>, line 4)

Saving the meeting matrix for later

In [14]:
np.save('../data/matrix/matrix222', meeting_matrix)

# OLD

#### Runnig time measurement

In [ ]:
# Getting the start time
start_time = time.time() 
print('Strat time: ' + time.ctime())

In [ ]:
# Getting the end time
end_time = time.time() 

# Getting thr run time in hours
run_time = (end_time - start_time) / 3600

# Getting the minutes and seconds
hours = int(run_time)
minutes = int((run_time - hours) * 60)
seconds  = int((((run_time - hours) * 60) - minutes)*60)

print('End time: ' + time.ctime())
print('Run time: ' + str(hours) + ' hours ' + str(minutes) + ' minutes ' + str(seconds) + ' seconds')